<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#KNN情感分析" data-toc-modified-id="KNN情感分析-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>KNN情感分析</a></span><ul class="toc-item"><li><span><a href="#KNN-算法" data-toc-modified-id="KNN-算法-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>KNN 算法</a></span><ul class="toc-item"><li><span><a href="#数据预处理" data-toc-modified-id="数据预处理-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>数据预处理</a></span><ul class="toc-item"><li><span><a href="#jieba-分词" data-toc-modified-id="jieba-分词-1.1.1.1"><span class="toc-item-num">1.1.1.1&nbsp;&nbsp;</span>jieba 分词</a></span></li><li><span><a href="#提取特征" data-toc-modified-id="提取特征-1.1.1.2"><span class="toc-item-num">1.1.1.2&nbsp;&nbsp;</span>提取特征</a></span></li><li><span><a href="#划分数据集" data-toc-modified-id="划分数据集-1.1.1.3"><span class="toc-item-num">1.1.1.3&nbsp;&nbsp;</span>划分数据集</a></span></li><li><span><a href="#特征展示" data-toc-modified-id="特征展示-1.1.1.4"><span class="toc-item-num">1.1.1.4&nbsp;&nbsp;</span>特征展示</a></span></li><li><span><a href="#训练模型" data-toc-modified-id="训练模型-1.1.1.5"><span class="toc-item-num">1.1.1.5&nbsp;&nbsp;</span>训练模型</a></span></li><li><span><a href="#测试模型" data-toc-modified-id="测试模型-1.1.1.6"><span class="toc-item-num">1.1.1.6&nbsp;&nbsp;</span>测试模型</a></span></li><li><span><a href="#分析数据" data-toc-modified-id="分析数据-1.1.1.7"><span class="toc-item-num">1.1.1.7&nbsp;&nbsp;</span>分析数据</a></span></li></ul></li></ul></li></ul></li></ul></div>

# KNN情感分析

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_excel(r"../data/TrainData.xlsx")

In [3]:
data.head()

,Comment,Sentiment
0,陈姥姥和何老师讲的特别棒，收获颇丰,1
1,很好很贴心！,1
2,有点敷衍了，内容少，连贯性一般，差强人意,0
3,视频讲解非常详细,1
4,没给代码，课件也没有,0


## KNN 算法

### 数据预处理
#### jieba 分词

In [4]:
# import jieba
# def chinese_word_cut(my_text):
#     jieba.load_userdict(dict_file)
#     jieba.initialize()
#     return " ".join(jieba.cut(my_text))

# data['cut_comment'] = data.Comment.apply(chinese_word_cut)

import jieba
import jieba.posseg as psg
import re

dict_file = r"../stop_dic/user_dict.txt"
stop_file = r"../stop_dic/hit_stopwords.txt"

def chinese_word_cut(my_text):  # my_text是要进行分词、去除停用词的txt文件
    jieba.load_userdict(dict_file)
    jieba.initialize()
    
    data_cut = jieba.lcut(my_text, cut_all=False)
    # 创建一个stop_word列表，读取停用词文档，将停用词添加到列表中
    stop_words_list = []
    with open(r'../stop_dic/hit_stopwords.txt', 'r', encoding='utf-8') as fp:
        for line in fp:
            if len(line) > 0:
                stop_words_list.append(line.strip())  # 将字符串前后的空格去掉
                
    # .新建一个列表，用于存放将分析文档库与停用词库对比后去除停用词的字符串
    data_result = []
    for i in data_cut:
        if i not in stop_words_list:
            i = i.replace(' ', '')
            i = i.replace('\n', '')
            if i != '':
                data_result.append(i)
    text = ' '.join(data_result).replace('\n', ' ')
#     return " ".join(jieba.cut(my_text))
    return text
data['cut_comment'] = data.Comment.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\20866\AppData\Local\Temp\jieba.cache
Loading model cost 0.559 seconds.
Prefix dict has been built successfully.


In [5]:
data.head()

,Comment,Sentiment,cut_comment
0,陈姥姥和何老师讲的特别棒，收获颇丰,1,陈 姥姥 老师 讲 特别 棒 收获 颇丰
1,很好很贴心！,1,很 好 很 贴心
2,有点敷衍了，内容少，连贯性一般，差强人意,0,有点 敷衍 内容 少 连贯性 差强人意
3,视频讲解非常详细,1,视频 讲解 非常 详细
4,没给代码，课件也没有,0,没 代码 课件 没有


#### 提取特征

In [6]:
from sklearn.feature_extraction.text import CountVectorizer  

In [7]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file, encoding='utf-8') as fp:
        stopwords = fp.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

stop_words_file = r"../stop_dic/hit_stopwords.txt"
stopwords = get_custom_stopwords(stop_words_file)
vect = CountVectorizer(max_df=0.8,
                      min_df=5,
                      token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                      stop_words=stopwords)
# 将文本文档集合转换为令牌计数矩阵。

#### 划分数据集

In [8]:
X = data['cut_comment']
Y = data.Sentiment

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=22)
# train_test_split() 将数组或矩阵分割为随机的训练和测试子集。
# random_state()在应用分割之前控制应用于数据的变换。传递一个int，用于跨多个函数调用的可重复输出。
# test_size : 表示测试比例是20%

#### 特征展示

In [9]:
# 展示训练数据的特征
test = pd.DataFrame(vect.fit_transform(X_train).toarray(), columns=vect.get_feature_names_out()) 
# 是get_feature_names_out，不是get_feature_names 新版本和旧版本不一样 
# get_feature_names_out获取转换的输出特性名。
test.head()

D:\SoftwareInstall\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


,app,c语言,good,java,kmp,linux,mooc,nice,oj,ok,...,非常适合,预习,预期,颇丰,题目,风格,风趣,高效,魅力,鼓励
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 训练模型

In [10]:
from sklearn.neighbors import KNeighborsClassifier

In [11]:
knn = KNeighborsClassifier(n_neighbors=5)  # weights='distance'
# Classifier implementing the k-nearest neighbors vote.分类器实现k近邻投票。
# 'distance':通过距离的倒数来加权点。在这种情况下，较近的查询点邻居比较远的邻居具有更大的影响。
# n_neighbors 每个样本所需的邻居数。默认值是传递给构造函数的|值。

X_train_vect = vect.transform(X_train)  # 将文本转换为n个gram计数的稀疏矩阵。

knn.fit(X_train_vect, Y_train)
train_score = knn.score(X_train_vect, Y_train)
# 返回给定测试数据和标签的平均准确度。在多标签分类中，这是子集精度，
# 这是一个苛刻的度量标准，因为您需要对每个样本的每个标签集进行正确预测。

train_score

D:\SoftwareInstall\anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.9291666666666667

#### 测试模型

In [12]:
# 模型在测试数据上的效果
X_test_vect = vect.transform(X_test)
print(knn.score(X_test_vect, Y_test))

0.9083333333333333


D:\SoftwareInstall\anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


#### 分析数据

In [13]:
data = pd.read_excel(r"../data/TestData_1.xlsx")

In [14]:
# data = pd.read_excel(r"../data/操作系统A.xlsx")
# data.head()

In [15]:
data['cut_comment'] = data.Comment.apply(chinese_word_cut)

In [16]:
X = data['cut_comment']
X_vec = vect.transform(X)
knn_result = knn.predict(X_vec)

data['knn_sentiment'] = knn_result
data.head()

D:\SoftwareInstall\anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


,Comment,cut_comment,knn_sentiment
0,老师授课很详细很具体很清楚,老师 授课 很 详细 很 具体 很 清楚,1
1,没听明白和没理解的可以反复学习,没 听 明白 没 理解 反复 学习,1
2,课程内容十分充实，老师用心准备，详略得当张弛有度，一口气学一单元不觉得累，我觉得收获很大。华...,课程内容 十分 充实 老师 用心 准备 详略 得当 张弛 有度 一口气 学一 单元 不 觉得...,1
3,零基础学习非常棒，详细的讲解，有效的练习,零 基础 学习 非常 棒 详细 讲解 有效 练习,1
4,好好，教的好得很,好好 教 好 很,1


In [17]:
# data.to_excel(r"../data/操作系统A.xlsx", index=False)
data.to_excel(r"../data/TestData_1.xlsx", index=False)
print('done')

done
